Creazione del pacchetto:



In [ ]:
mkdir python_catkin #creo cartella per pacchetto
mkdir src 
catkin_make #creo il pacchetto
cd src      
catkin_create_pkg tutorial rospy std_msgs #creo un pacchetto python con dipendenze std_msgs

In tutorials creo una cartella chiamata "scripts" al pari di src

In "scripts" creo: i file publisher_node.py, subscriber_node.py, service_client_node.py, service_server_node.py


In [ ]:
#publisher node

#!/usr/bin/envcd ..
import rospy
from std_msgs.msg import String

def talk_to_me():
    pub = rospy.Publisher('talking_topic', String, queue_size = 10) #declaring a pub that publishes a String on the topic called "talking_topic" 
    rospy.init_node('publisher_node', anonymous=True) #The node is called "publisher_node". Anonimous fa si che se ho più pub gli da un nome univoco con numero sequenziale
    rate = rospy.Rate(1)
    rospy.loginfo("Publisher node started to publish") 
    while not rospy.is_shutdown():
        msg = "Hey i'm your first pubblisher man!: %s" % rospy.get_time() 
        pub.publish(msg)    #it's a method of a pubblisher!!!
        rospy.loginfo("Io ho pubblicato man!")
        rate.sleep()

if __name__ == '__main__':
    try:
        talk_to_me()
    except rospy.ROSInterruptException:
        pass



In [ ]:
#subscriber node

#!/usr/bin/env
import rospy
from std_msgs.msg import String

def callback(incoming_data):#incoming_data è nome arbitrario...
    rospy.loginfo("RECEIVED DATA: %s", incoming_data.data) #dal manuale ho string msg ha campo "data"
    
def listener():
    rospy.init_node('subscriber_node', anonymous=True)
    rospy.Subscriber('talking_topic', String, callback) #a differenza del pub qui non devo fare il sub = rospy.Subscriber
    rospy.spin()

if __name__ == '__main__':
    try:
        listener()
    except rospy.ROSInterruptException:
        pass





When using services, i need to define both input and the response message. This is done through an ".srv" file... 

In [ ]:
#with this i'm defining outputs and inputs of the service
int32 a #input
int32 b #input
---
int32 result #output

In [ ]:
#service_server_node

 #!/usr/bin/env python
import rospy
from tutorials.srv import multiplier, multiplierResponse #importo i tipi di messaggio

def callback(request):
    return multiplierResponse(request.a*request.b)

def multiply():
    rospy.init_node("multiplier_service") #inizializzo il nodo
    service = rospy.Service("multiplier", multiplier, callback)
    rospy.spin()

if __name__ == '__main__':
    multiply()

In [ ]:
 #server_client_node 
 
 #!/usr/bin/env python
import rospy
from tutorials.srv import multiplier, multiplierResponse #importo i tipi di messaggio

def multiplier_client(x,y):
    rospy.init_node("client_node")
    rospy.wait_for_service("multiplier")
    rate = rospy.Rate(1)
    while not rospy.is_shutdown():
        try:
            multiply_two_ints = rospy.ServiceProxy("multiplier",multiplier) #name of the service & type of message (the name of the srv file in witch we defined the message.in this case message.srv => message)
            response = multiply_two_ints(x,y)#multiply_two_ints è instanza di servizio. Qui lo chiamo passandogli x e y
            rospy.loginfo(response.result)
            rate.sleep()
        except rospy.ServiceException as e:
            print('service call fail!')


if __name__ == '__main__':
    multiplier_client(7, 2)

L'aggiunta di codice .py (anche c++) richiede l'aggiornamento del CMakeLists.txt del pacchetto:

In [ ]:
#qui aggiungo i nomi degli scripts e vale sia per pub/sub sia i service
catkin_install_python(PROGRAMS scripts/publisher_node.py scripts/subscriber_node.py scripts/service_server_node.py scripts/service_client_node.py
  DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
)

In [ ]:
#Per i servizi devo anche scommentare i seguenti:
#-------------------------------------------------------
find_package(catkin REQUIRED COMPONENTS
  rospy
  std_msgs
  message_generation #aggiungo questa "message_generation" 
)
#-------------------------------------------------------
#scrivo al posto di Service1.srv il nome dei miei servizi 
 add_service_files(
   FILES
   multiplier.srv
 )
#-------------------------------------------------------
#decommento i generate_message
 generate_messages(
   DEPENDENCIES
   std_msgs  # Or other packages containing msgs
 )
#-------------------------------------------------------
 

Solo perchè ho introdotto i servizi, modifico il package.xml scommentando le seguenti righe:

In [ ]:
#come indicato nella pagina di ROS tutorial sui servizi

<build_depend>message_generation</build_depend> 
<exec_depend>message_runtime</exec_depend> 


E' importante ad ogni terminale fare un source devel/setup.bash a meno che non si inserisca nel bashrc l'apposita riga (che non ricordo) per far si che ad ogni terminale faccia il source in automatico...

E' importante ad ogni modifica fatta ai file fare un catkin_make lanciata dal workspace ("python_catkin" nell'esempio)